<a href="https://colab.research.google.com/github/emiliagyr/EstadisticaExperimental/blob/main/Colab/Ejercicio5_ANOVA_Solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pingouin
!pip install pyreadstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import seaborn as sns
import pingouin as pg

#Enunciado
La intervención Video-feedback Intervention to promote Positive Parenting and Sensitive Discipline (VIPP-SD) tiene como objetivo incrementar la sensibilidad materna mediante 7 sesiones en las que se filman y discuten distintas interacciones de la diada madre-niño. Debido a que esta intervención ha sido probada solamente en los Países Bajos, un grupo de investigación de la PUCP deseaba evaluar si se tendrían resultados positivos en una muestra piloto de madres peruanas. Para ello se trabajó con 120 diadas madre niño y se las evaluó mediante la escala de sensibilidad de Ainsworth en 4 momentos (antes de la intervención, después de la intervención, 6 meses después del final de la intervención y 12 meses después del final de la intervención).

a. El grupo de investigación buscaba identificar si habría diferencias en la sensibilidad materna en los distintos momentos a raíz de la intervención. Asuma normalidad en la distribución de las mediciones.

In [ ]:
#VD: cantidad de palabras recordadas
#VI: condiciones
    #N1: Solo palabras
    #N2: Palabras + imágenes
    #N3: Palabras + imágenes + audio

#Diseño: Intrasujeto

##Hipotesis
HT: 3 medidas son diferentes

HO: N1 = N2 = N3 = N4

H1: Al menos 2 medidas son diferentes

#1. Leer los datos del documento .sav (SPSS) o .csv (Excel)

In [6]:
df = pd.read_spss("Word_recall_2.sav")
df

,word,wordpic,wordpicsound
0,62.0,70.0,82.0
1,63.0,68.0,68.0
2,65.0,61.0,72.0
3,68.0,75.0,88.0
4,69.0,72.0,80.0
5,71.0,77.0,80.0
6,78.0,82.0,87.0
7,75.0,73.0,79.0
8,70.0,77.0,82.0
9,71.0,76.0,84.0


In [7]:
df['subject'] = df.index // 1
df

,word,wordpic,wordpicsound,subject
0,62.0,70.0,82.0,0
1,63.0,68.0,68.0,1
2,65.0,61.0,72.0,2
3,68.0,75.0,88.0,3
4,69.0,72.0,80.0,4
5,71.0,77.0,80.0,5
6,78.0,82.0,87.0,6
7,75.0,73.0,79.0,7
8,70.0,77.0,82.0,8
9,71.0,76.0,84.0,9


In [8]:
melt_df = pd.melt(
    df, value_name="cantidad_palabras", var_name="condición", id_vars=["subject"]
)
melt_df

,subject,condición,cantidad_palabras
0,0,word,62.0
1,1,word,63.0
2,2,word,65.0
3,3,word,68.0
4,4,word,69.0
5,5,word,71.0
6,6,word,78.0
7,7,word,75.0
8,8,word,70.0
9,9,word,71.0


#2. Estadisticos descriptivos

In [9]:
df.describe()

,word,wordpic,wordpicsound,subject
count,11.000000,11.000000,11.000000,11.000000
mean,68.363636,72.818182,79.909091,5.000000
std,5.518564,5.600325,5.957425,3.316625
min,60.000000,61.000000,68.000000,0.000000
25%,64.000000,70.000000,78.000000,2.500000
50%,69.000000,73.000000,80.000000,5.000000
75%,71.000000,76.500000,83.000000,7.500000
max,78.000000,82.000000,88.000000,10.000000


# 3. Normalidad

In [11]:
pg.normality(data=melt_df, dv="cantidad_palabras", group="condición", method="shapiro")

,W,pval,normal
condición,,,
word,0.971110,0.897508,True
wordpic,0.964782,0.829553,True
wordpicsound,0.946118,0.594840,True


# 4. Análisis de medias repetidas

Prueba de esfericidad de Mauchly

In [13]:
pg.sphericity(melt_df)

SpherResults(spher=False, W=0.15600531836312598, chi2=57.593820587514436, dof=2, pval=3.1164543058960006e-13)

No se asume esfericidad, por lo tanto se usa prueba de Greenhouse-Geisser

In [15]:
pg.rm_anova(data=melt_df, dv="cantidad_palabras", within="condición", subject="subject", correction='GG')

,Source,ddof1,ddof2,F,p-unc,ng2,eps
0,condición,2,20,35.162857,2.832604e-07,0.43391,0.825804


In [16]:
pg.pairwise_tests(data=melt_df, dv="cantidad_palabras", within="condición", padjust="bonf", subject="subject", effsize="cohen")

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,cohen
0,condición,word,wordpic,True,True,-3.534430,10.0,two-sided,0.005407,0.016220,bonf,9.976,-0.801236
1,condición,word,wordpicsound,True,True,-6.891597,10.0,two-sided,0.000042,0.000127,bonf,603.813,-2.010637
2,condición,wordpic,wordpicsound,True,True,-6.014252,10.0,two-sided,0.000130,0.000389,bonf,230.087,-1.226455


In [18]:
#Para saber cuál es mayor volvemos a la media
df.describe()

,word,wordpic,wordpicsound,subject
count,11.000000,11.000000,11.000000,11.000000
mean,68.363636,72.818182,79.909091,5.000000
std,5.518564,5.600325,5.957425,3.316625
min,60.000000,61.000000,68.000000,0.000000
25%,64.000000,70.000000,78.000000,2.500000
50%,69.000000,73.000000,80.000000,5.000000
75%,71.000000,76.500000,83.000000,7.500000
max,78.000000,82.000000,88.000000,10.000000
